<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chapter-3" data-toc-modified-id="Chapter-3-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Chapter 3</a></span><ul class="toc-item"><li><span><a href="#Using-an-enum-to-represent-players" data-toc-modified-id="Using-an-enum-to-represent-players-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Using an enum to represent players</a></span></li><li><span><a href="#Using-Tuples-to-represent-points-of-a-Go-board" data-toc-modified-id="Using-Tuples-to-represent-points-of-a-Go-board-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Using Tuples to represent points of a Go board</a></span></li><li><span><a href="#Setting-Moves:-plays,-passes,-or-resigns" data-toc-modified-id="Setting-Moves:-plays,-passes,-or-resigns-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Setting Moves: plays, passes, or resigns</a></span></li><li><span><a href="#Encoding-Strings-of-stones-with-set" data-toc-modified-id="Encoding-Strings-of-stones-with-set-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Encoding Strings of stones with set</a></span></li><li><span><a href="#Creating-a-Go-Board-instance" data-toc-modified-id="Creating-a-Go-Board-instance-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Creating a Go Board instance</a></span></li><li><span><a href="#Creating-neighboring-points-for-liberties" data-toc-modified-id="Creating-neighboring-points-for-liberties-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Creating neighboring points for liberties</a></span></li><li><span><a href="#Utility-methods-for-placing-and-removing-stones" data-toc-modified-id="Utility-methods-for-placing-and-removing-stones-3.7"><span class="toc-item-num">3.7&nbsp;&nbsp;</span>Utility methods for placing and removing stones</a></span></li><li><span><a href="#Continuing-our-definition-of-place_stone" data-toc-modified-id="Continuing-our-definition-of-place_stone-3.8"><span class="toc-item-num">3.8&nbsp;&nbsp;</span>Continuing our definition of place_stone</a></span></li></ul></li></ul></div>

# GoTypes Jupyter Notebook

This notebook follows along with the book "Deep Learning and the Game of Go" by Max Pumperla and Kevin Ferguson. Some people have mentioned that I need to do a better job commenting out my code to explain intermediate python tactics. Thus, I will be commenting out many lines to not only completely understand code, but also sharpen my teaching skills. 

## Chapter 3

### Using an enum to represent players

<i>Insert into gotypes.py</i>

In [8]:
import enum 

Using enum with the different player classes is logical since enum comes with all kinds of useful goodies.

In [9]:
class Player(enum.Enum):
    black = 1
    white = 2
    
    @property
    def other(self):
        return Player.black if self == Players.white else Player.white

### Using Tuples to represent points of a Go board

<i>Insert into gotypes.py</i>

In [16]:
from collections import namedtuple

# This library is chosen to help with code readability.
# point.row is easier to read than point[0]

class Point(namedtuple("Point", "row col")):
    
    # This will allow us to look at the liberties of each stone
    def neighbors(self):
        return [Point(self.row - 1, self.col),
                Point(self.row + 1, self.col),
                Point(self.row, self.col - 1),
                Point(self.row, self.col + 1)]

### Setting Moves: plays, passes, or resigns

<i>Insert into goboard_slow.py</i>

In [15]:
import copy
from gotypes import Player

# This import is interesting, we are already coding
# above, I think I will make a new .py file and add it the current 
# working directory


class Move():
    
    # In the game of Go an action is defined as either a play, 
    # a pass or resign. Plays are defined as placing a stone.
    
    
    def __init__ (self,
                  point = None,
                  is_pass = False, 
                  is_resign = False):
        assert (point is not None) ^ is_pass ^ is_resign
        
        # The carrot here (^) is the pythonic XOR operator.
        # assert allows us to interupt the program unless 
        # one of these is true. Notice that if two or more are
        # true, then this will also break the line.
        
        @classmethod 
        def play(cls, point):
            return Move(point = point)
        
        @classmethod
        def pass_turn(cls):
            return Move(is_pass = True)
        
        @classmethod
        def resign(cls):
            return Move(is_resign = True)

### Encoding Strings of stones with set

<i>Insert into goboard_slow.py</i>

Tracking each stone's liberties would be computationally burdensome. Thus, we will be looking at connected groups of stones.

In [21]:
class GoString():  # <1>
    def __init__(self, color, stones, liberties):
        self.color = color
        self.stones = set(stones)
        self.liberties = set(liberties)

    def remove_liberty(self, point):
        self.liberties.remove(point)

    def add_liberty(self, point):
        self.liberties.add(point)

    def merged_with(self, go_string):  # <2>
        assert go_string.color == self.color
        combined_stones = self.stones | go_string.stones
        return GoString(
            self.color,
            combined_stones,
            (self.liberties | go_string.liberties) - combined_stones)

    @property
    def num_liberties(self):
        return len(self.liberties)

    def __eq__(self, other):
        return isinstance(other, GoString) and \
            self.color == other.color and \
            self.stones == other.stones and \
            self.liberties == other.liberties

### Creating a Go Board instance

<i>Insert into goboard_slow.py</i>

In [22]:
class Board():
    
    #We initialize an empty grid with the number of rows and columns
    def __init__(self, num_rows, num_cols):
        self.num_rows = num_rows
        self.num_cols = num_cols
        self._grid = {}

### Creating neighboring points for liberties

<i>Insert into goboard_slow.py</i>

In [25]:
def place_stone(self, player, point):
    assert self.is_on_grid(point)
    assert self._grid.get(point)
    adjacent_same_color = []
    adjacent_opposite_color = []
    liberties = []
    for neighbor in point.neighbors():
        if not self.is_on_grid(neighbors):
            continue
        neighbor_string = self._grid.get(neighbor)
        if neighbor_string is None:
            liberties.append(neighbor)
        elif nighbor_string.color == player:
            if neighbor_string not in adjacent_same_color:
                adjacent_same_color.append(neighbor_string)
        
        # Not sure how I feel abou these weird nested ifs
        # Probably best to fix it once I have the code
        # Running
        
        else: 
            if neighbor_string not in adjacent_opposite_color:
                adjacent_opposite_color.append(neighbor_string)
            
    new_string = GoString(player, [point], liberties)

### Utility methods for placing and removing stones 

In [30]:
def is_on_grid(self, point):
    return 1 <= point.row <= self.num_rows and \
        1 <= point.col <= self.num_cols

# Returns the content of a point on the board: a player if a stone
# is on that point, or else None

def get(self, point):
    string = self._grid.get(point)
    if string is None:
        return None
    return string.color

# Returns the entire string of stones at a point: a GoString if a 
# stone is on that point, or else None

def get_go_string(self, point):
    string = self._grid.get(point)
    if string is None:
        return None
    return String

### Continuing our definition of place_stone

Oh, I really don't like 